# Certifying and Removing Disparate Impact

This notebook apples the algorithm described in [Certifying and removing disparate impact](https://dl.acm.org/doi/10.1145/2783258.2783311) by Feldman et al., as implemented by the [AI Fairness 360 library](https://aif360.readthedocs.io/) from IBM.

This is a pre-processing algorithm that works by adjusting the distributions of the features conditional on the protected attribute to be equal, so that a subsequently trained model can't discriminate.

In [ ]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
)
from helpers.plot import group_box_plots
from sklearn.neural_network import MLPClassifier  # noqa

In [ ]:
from helpers import export_plot

## Load data

We have committed preprocessed data to the repository for reproducibility and we load it here. Check out the preprocessing notebook for details on how this data was obtained.

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "adult"

train_oh = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val_oh = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test_oh = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

`aif360` uses the following custom dataset objects

In [ ]:
train_sds = StandardDataset(
    train_oh,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val_oh,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test_oh,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
index = train_sds.feature_names.index("sex")

## Train unfair model

For maximum reproducibility we load the baseline model from disk, but the code used to train can be found in the baseline model notebook.

In [ ]:
bl_model = joblib.load(artifacts_dir / "models" / "finance" / "baseline.pkl")

bl_test_probs = bl_model.predict_proba(test_sds.features)[:, 1]
bl_test_pred = bl_test_probs > 0.5

## Perform intervention

We repair the dataset using the `DisparateImpactRemover`.

In [ ]:
di = DisparateImpactRemover(repair_level=1.0)

train_repd = di.fit_transform(train_sds)
train_repd_X = np.delete(train_repd.features, index, axis=1)
train_repd_y = train_repd.labels.flatten()

test_repd = di.fit_transform(test_sds)
test_repd_X = np.delete(test_repd.features, index, axis=1)
test_repd_y = test_repd.labels.flatten()

## Train model on fair data

We use the same architecture, but the repaired data. Once again we load a trained model for reproducibility, but the code used to train the model can be found below.

In [ ]:
model = joblib.load(artifacts_dir / "models" / "finance" / "feldman.pkl")

In [ ]:
# model = MLPClassifier(hidden_layer_sizes=(100, 100), early_stopping=True)
# model.fit(train_repd_X, train_repd_y)

test_probs = model.predict_proba(test_repd_X)[:, 1]
test_pred = test_probs > 0.5

## Analyse unfairness and accuracy

We measure the accuracy and fairness in baseline and compare it to the corrected model.

In [ ]:
bl_acc = bl_model.score(test_oh.drop(columns="salary"), test_oh.salary)
bl_dpd = demographic_parity_difference(
    test_oh.salary,
    bl_test_pred,
    sensitive_features=test_oh.sex,
)
bl_dpr = demographic_parity_ratio(
    test_oh.salary,
    bl_test_pred,
    sensitive_features=test_oh.sex,
)

acc = model.score(test_repd_X, test_oh.salary)
dpd = demographic_parity_difference(
    test_oh.salary,
    test_pred,
    sensitive_features=test_oh.sex,
)
dpr = demographic_parity_ratio(
    test_oh.salary,
    test_pred,
    sensitive_features=test_oh.sex,
)

print(f"Baseline accuracy: {bl_acc:.3f}")
print(f"Accuracy: {acc:.3f}\n")

print(f"Baseline demographic parity difference: {bl_dpd:.3f}")
print(f"Demographic parity difference: {dpd:.3f}\n")

print(f"Baseline demographic parity ratio: {bl_dpr:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

We can visualise the disparity between men and women with a box plot of the scores.

In [ ]:
dp_box = group_box_plots(
    np.concatenate([bl_test_probs, test_probs]),
    np.tile(test_oh.sex.map(lambda x: "Male" if x else "Female"), 2),
    groups=np.concatenate(
        [np.zeros_like(bl_test_probs), np.ones_like(test_probs)]
    ),
    group_names=["Baseline", "Feldman"],
    title="Distribution of scores by sex",
    xlabel="Score",
    ylabel="Model",
    
)
dp_box

In [ ]:
export_plot(dp_box, "feldman-dp.json")